In [1]:
import numpy as np
import pandas as pd
import MetaTrader5 as mt5

import statsmodels.api as sm
from pandas.tseries.offsets import BDay

import csv
import time
import math
from datetime import datetime, timedelta

import requests
import json

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

colors = cycler('color', ['669FEE', '66EE91', '9988DD', 'EECC55', '88BB44', 'FFBBBB'])
plt.rc('figure', facecolor='313233')
plt.rc('axes',  facecolor='313233', edgecolor='none', axisbelow=True, grid=True, prop_cycle=colors, labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor='313233', edgecolor='313233')
plt.rc('text')

In [3]:
# Twitter notification keys

chat_id = "6208180231"
api_key = "6274819941:AAF8U6FnGv5A0DcSBNXCpnPCJPciwSww8-A"

In [4]:
def send_telegram_message(message: str, chat_id: str, api_key: str,):
    responses = {}

    proxies = None
    headers = {'Content-Type': 'application/json',
                'Proxy-Authorization': 'Basic base64'}
    data_dict = {'chat_id': chat_id,
                 'text': message,
                 'parse_mode': 'HTML',
                 'disable_notification': True}
    data = json.dumps(data_dict)
    url = f'https://api.telegram.org/bot{api_key}/sendMessage'
    
    requests.packages.urllib3.disable_warnings()
    response = requests.post(url,
                             data=data,
                             headers=headers,
                             proxies=proxies,
                             verify=False)
    return response

In [5]:
def get_bars(symbol, start_date, end_date, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bars = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    df_bars = pd.DataFrame(bars)
    df_bars["time"] = pd.to_datetime(df_bars["time"], unit="s")
    return df_bars

In [6]:
def get_latest_bar(symbol, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bar = mt5.copy_rates_from(
        symbol, 
        timeframe, 
        datetime.now(), 
        1)  # number of bars
    df_bar = pd.DataFrame(bar)
    df_bar["time"] = pd.to_datetime(df_bar["time"], unit="s")
    return df_bar

In [7]:
def get_prices(location):
    mt5.initialize()

    symbol_list = []
    for symbol in mt5.symbols_get():
        if location in symbol.path:
            symbol_list.append(symbol)
            
    return symbol_list

In [8]:
def new_bar_event(symbol, last_bar):
    mt5.initialize()
    
    # Fixing a formatting error when getting last_bar['time'].iloc[0]
    date_str = last_bar['time'].iloc[0]
    date_format = '%Y-%m-%d %H:%M:%S'

    last_bar_time_formatted = datetime.strptime(date_str, date_format)
    while True:
        new_bar = get_latest_bar(symbol)        
        if new_bar['time'].iloc[0] != last_bar_time_formatted:
            return new_bar
        
        time.sleep(10)

In [9]:
def get_num_of_lots(name, leverage=30, risk=0.01):
    ask_per_fraction = (mt5.symbols_get(name)[0].ask * mt5.symbols_get(name)[0].trade_contract_size) * mt5.symbols_get(name)[0].volume_min
    ask_per_fraction_m = ask_per_fraction / leverage
    capital_at_risk = mt5.account_info().balance * risk
    
    order_size = math.floor(capital_at_risk / ask_per_fraction_m) * mt5.symbols_get(symbol)[0].volume_min
    return  order_size

In [10]:
def find_filling_mode(symbol):
    mt5.initialize()
    
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}
 
        result = mt5.order_check(request)
            
        if result.comment == "Done":
            break
 
    return i

In [11]:
def open_long(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [12]:
def close_long(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [13]:
def open_short(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    return result

In [14]:
def close_short(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
        
    result = mt5.order_send(request)   # Sends order to broker
    return result  

In [15]:
# Storing order history
def save_to_order_history(result):
    order = pd.DataFrame(
        columns=[
            'retcode',
            'deal',
            'order',
            'volume',
            'price',
            'bid',
            'ask',
            'comment',
            'request_id',
            'retcode_external',
            'request.action',
            'request.magic',
            'request.order',
            'request.symbol',
            'request.volume',
            'request.price',
            'request.stoplimit',
            'request.sl',
            'request.tp',
            'request.deviation',
            'request.type',
            'request.type_filling',
            'request.type_time',
            'request.expiration',
            'request.comment',
            'request.position',
            'request.position_by'])

    new_row = {
            'retcode': result.retcode,
            'deal': result.deal,
            'order': result.order,
            'volume': result.volume,
            'price': result.price,
            'bid': result.bid,
            'ask': result.ask,
            'comment': result.comment,
            'request_id': result.request_id,
            'retcode_external': result.retcode_external,
            'request.action': result.request.action,
            'request.magic': result.request.magic,
            'request.order': result.request.order,
            'request.symbol': result.request.symbol,
            'request.volume': result.request.volume,
            'request.price': result.request.price,
            'request.stoplimit': result.request.stoplimit,
            'request.sl': result.request.sl,
            'request.tp': result.request.tp,
            'request.deviation': result.request.deviation,
            'request.type': result.request.type,
            'request.type_filling': result.request.type_filling,
            'request.type_time': result.request.type_time,
            'request.expiration': result.request.expiration,
            'request.comment': result.request.comment,
            'request.position': result.request.position,
            'request.position_by': result.request.position_by
        }

    df_new_row = pd.DataFrame([new_row])
    current_order = pd.concat([order, df_new_row], ignore_index=True)
        
    current_order.to_csv('order_history.csv', mode='a', index=False, header=False)


In [16]:
def draw_graph(data, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold):
    
    plt.plot(data)
    plt.plot(fast_ma, color='r')
    plt.plot(upper_threshold, color='y')
    plt.plot(lower_threshold, color='y')
    plt.show()
    
    plt.plot(data_vol)
    plt.plot(vol_threshold, color='y')
    plt.show()
    

In [17]:
def trade_engine(name, lot_size, data):
    mt5.initialize()
    
    # Get close and volume data
    data_close = data['close']
    data_vol = data['tick_volume']
    
    # Apply HP filter 
    try:
        filtered_data_close_cycle, filtered_data_close_trend = sm.tsa.filters.hpfilter(data_close)
        filtered_data_vol_cycle, filtered_data_vol_trend = sm.tsa.filters.hpfilter(data_vol)
    except ValueError:
        print("HP filter acting funny, skipping.")
        return
    
    # Construct volume mean and std
    window_vol = 100
    us500_vol_mean = filtered_data_vol_trend.rolling(window_vol).mean()
    us500_vol_std = filtered_data_vol_trend.rolling(window_vol).std()
    
    # Construct slow MA, slow STD, and fast MA
    window_slow = 50
    window_fast = 10
    slow_ma = filtered_data_close_trend.ewm(span=window_slow).mean()
    slow_std = filtered_data_close_trend.ewm(span=window_slow).std()
    fast_ma = filtered_data_close_trend.ewm(span=window_fast).mean()
    
    
    # Contruct trade parameters
    upper_threshold = slow_ma + (slow_std * 0.3)
    lower_threshold = slow_ma - (slow_std * 0.3)
        
    vol_threshold = us500_vol_mean + (us500_vol_std * 0.3)
      
    # Position type encoding
    LONG = 0
    SHORT = 1
         
    if not mt5.positions_get():
        if (fast_ma.iloc[-2] <= upper_threshold.iloc[-2]) and (fast_ma.iloc[-1] >= upper_threshold.iloc[-1]) and data_vol.iloc[-1] >= vol_threshold.iloc[-1]:
            # OPEN LONG POSITION
            result = open_long(name, lot_size)
            save_to_order_history(result)
            send_telegram_message(f'Openned Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'Openned Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            draw_graph(data_close, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold)
            
            

        elif (fast_ma.iloc[-2] >= lower_threshold.iloc[-2]) and (fast_ma.iloc[-1] <= lower_threshold.iloc[-1]) and data_vol.iloc[-1] >= vol_threshold.iloc[-1]:
            # OPEN SHORT POSITION
            result = open_short(name, lot_size)
            save_to_order_history(result)
            send_telegram_message(f'Openned Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
            print(f'Openned Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
            draw_graph(data_close, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold)
            
    elif mt5.positions_get():
        position_type = mt5.positions_get()[0].type
        ticket = mt5.positions_get()[0].ticket
        
        if position_type == LONG:     
            if (fast_ma.iloc[-2] >= slow_ma.iloc[-2]) and (fast_ma.iloc[-1] < slow_ma.iloc[-1]):
                # CLOSE LONG POSITION AT A GAIN
                result = close_long(name, lot_size, ticket)
                save_to_order_history(result)
                send_telegram_message(f'Closed Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
                print(f'Closed Long Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
                draw_graph(data_close, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold)
                
            elif (mt5.positions_get()[0].price_open - data_close.iloc[-1]) >= 3 * (slow_ma.iloc[-1] + slow_std.iloc[-1]):
                # CLOSE LONG POSITION: BLACK SWAN
                result = close_long(name, lot_size, ticket)
                save_to_order_history(result)
                send_telegram_message(f'Black Swanned Long at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
                print(f'Black Swanned Long at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
                draw_graph(data_close, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold)
                
        elif position_type == SHORT:        
            if (fast_ma.iloc[-2] <= slow_ma.iloc[-2]) and (fast_ma.iloc[-1] > slow_ma.iloc[-1]):
                # CLOSE SHORT POSITION AT A GAIN
                result = close_short(name, lot_size, ticket)
                save_to_order_history(result)
                send_telegram_message(f'Closed Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
                print(f'Closed Short Positions at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
                draw_graph(data_close, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold)
            elif (data_close.iloc[-1] - mt5.positions_get()[0].price_open) >= 3 * (slow_ma.iloc[-1] + slow_std.iloc[-1]):
                # CLOSE SHORT POSITION: BLACK SWAN
                result = close_short(name, lot_size, ticket)
                save_to_order_history(result)
                send_telegram_message(f'Black Swanned Short at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.', chat_id, api_key)
                print(f'Black Swanned Short at: {data_close.iloc[-1]}, Account Balance is {mt5.account_info().balance}.')
                draw_graph(data_close, fast_ma, upper_threshold, lower_threshold, data_vol, vol_threshold)

In [18]:
##################################################
# Set Parameters

end_date = datetime.now()
start_date = end_date - BDay(1)

mt5.initialize()

# MOST IMPORTANT PARAMETERS
symbol = 'US500'
#lot_size = mt5.symbols_get(symbol)[0].volume_min * 3

symbol_csv = symbol + '.csv'

##################################################

In [19]:
##################################################
# Generate a days worth of market data and put it into a csv

mt5.initialize()
symbol_object = mt5.symbols_get(symbol)
symbol_name = symbol_object[0].name

symbol_data = get_bars(
    symbol_name, 
    start_date, 
    end_date)

symbol_data.to_csv(symbol_csv, index=False)

data = pd.read_csv(symbol_csv)

In [ ]:
##################################################
# Trade Loop

while True:
    last_bar = data.tail(1)
    new_bar = new_bar_event(symbol_name, last_bar)
    
    new_bar.to_csv(symbol_csv, mode='a', index=False, header=False)
    data = pd.read_csv(symbol_csv)

    lot_size = get_num_of_lots(symbol, leverage=20, risk=0.1)
    print(f'NEW BAR: {new_bar["time"].iloc[0]}')
    
    trade_engine(symbol_name, lot_size, data)
    
##################################################

NEW BAR: 2023-03-20 23:35:00
